In [1]:
import pandas as pd
import numpy as np
import mysql.connector as conn
import warnings
warnings.filterwarnings('ignore')

In [2]:
mydb=conn.connect(host='localhost', user='root', password='password',auth_plugin='mysql_native_password')

In [3]:
cur=mydb.cursor(buffered=True)

In [4]:
cur.execute('use music_store')

In [5]:
cur.execute('show tables')
table_list=[table[0] for table in cur.fetchall()]

In [6]:
table_list

['album',
 'artist',
 'customer',
 'employee',
 'genre',
 'invoice',
 'invoice_line',
 'media_type',
 'playlist',
 'playlist_track',
 'track']

In [7]:
df=pd.read_sql_query('select * from album',mydb)
df.head()

,album_id,title,artist_id
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [8]:
print(df.shape)
print(df.size)

(347, 3)
1041


In [9]:
cur.execute('select * from album')
cur.description

[('album_id', 3, None, None, None, None, 1, 32768, 63),
 ('title', 252, None, None, None, None, 1, 16, 255),
 ('artist_id', 3, None, None, None, None, 1, 32768, 63)]

In [10]:
column=[i[0] for i in cur.description]
print(column)

['album_id', 'title', 'artist_id']


In [11]:
#Checking for datatypes
for table in table_list:
    print('\nTable:',table)
    column_info=cur.execute(f'''select column_name, data_type from Information_schema.columns where table_schema=
    "music_store" and table_name = "{table}" ''')
    for i in cur.fetchall():
        print(i)


Table: album
('album_id', 'int')
('title', 'text')
('artist_id', 'int')

Table: artist
('artist_id', 'int')
('name', 'text')

Table: customer
('customer_id', 'int')
('first_name', 'text')
('last_name', 'text')
('company', 'text')
('address', 'text')
('city', 'text')
('state', 'text')
('country', 'text')
('postal_code', 'text')
('phone', 'text')
('fax', 'text')
('email', 'text')
('support_rep_id', 'int')

Table: employee
('employee_id', 'int')
('last_name', 'text')
('first_name', 'text')
('title', 'text')
('reports_to', 'int')
('levels', 'text')
('birthdate', 'text')
('hire_date', 'text')
('address', 'text')
('city', 'text')
('state', 'text')
('country', 'text')
('postal_code', 'text')
('phone', 'text')
('fax', 'text')
('email', 'text')

Table: genre
('genre_id', 'int')
('name', 'text')

Table: invoice
('invoice_id', 'int')
('customer_id', 'int')
('invoice_date', 'text')
('billing_address', 'text')
('billing_city', 'text')
('billing_state', 'text')
('billing_country', 'text')
('billing

In [12]:
#Checking for null values
for table in table_list:
    print('\nTable: ',table)
    df_table=pd.read_sql_query(f'select * from {table}',mydb)
    print(df_table.isnull().sum())


Table:  album
album_id     0
title        0
artist_id    0
dtype: int64

Table:  artist
artist_id    0
name         0
dtype: int64

Table:  customer
customer_id       0
first_name        0
last_name         0
company           0
address           0
city              0
state             0
country           0
postal_code       0
phone             0
fax               0
email             0
support_rep_id    0
dtype: int64

Table:  employee
employee_id    0
last_name      0
first_name     0
title          0
reports_to     0
levels         0
birthdate      0
hire_date      0
address        0
city           0
state          0
country        0
postal_code    0
phone          0
fax            0
email          0
dtype: int64

Table:  genre
genre_id    0
name        0
dtype: int64

Table:  invoice
invoice_id             0
customer_id            0
invoice_date           0
billing_address        0
billing_city           0
billing_state          0
billing_country        0
billing_postal_code    0
t

In [13]:
# 1. Who is the senior most employee based on job title?
emp=pd.read_sql_query('select * from employee',mydb)
emp.head(2)

,employee_id,last_name,first_name,title,reports_to,levels,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
0,1,Adams,Andrew,General Manager,9,L6,18-02-1962 00:00,14-08-2016 00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1,L4,08-12-1958 00:00,01-05-2016 00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com


In [14]:
cur.execute('select concat(first_name," ", last_name) as full_name,levels from employee order by levels desc limit 1')
cur.fetchall()

[('Andrew Adams', 'L6')]

In [15]:
#2. Which countries have the most Invoices?
cur.execute('select count(*) as Number_of_invoices, billing_country from invoice group by billing_country order by Number_of_invoices desc limit 1')
cur.fetchall()

[(131, 'USA')]

In [16]:
# Q3) What are top 3 values of total invoice?
invoice=pd.read_sql_query('select * from invoice',mydb)
invoice.head(2)

,invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
0,1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
1,2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.90


In [17]:
cur.execute('select total from invoice order by total desc limit 3')
cur.fetchall()

[(23.759999999999998,), (19.8,), (19.8,)]

In [18]:
# Q4) Which city has the best customers? We would like to throw a promotional music festival in the city we made the most money. 
# Write a query that returns one city that has the highest sum of invoices totals. Return both the city name & sum of all invoices
# totals
cur.execute('select billing_city,sum(total) as invoice_total from invoice group by billing_city order by invoice_total desc limit 1')
cur.fetchall()

[('Prague', 273.24000000000007)]

In [19]:
# Q5) Who is the best customer? The customer who has spent the most money will be declared as the best customer.
# Write a query that returns the person who has spent the most money.
cur.execute('select concat(c.first_name, " ", c.last_name)as full_name, sum(i.total) as total from customer as c join invoice as i on c.customer_id=i.customer_id group by i.customer_id order by total desc limit 1')
cur.fetchall()

[('FrantiÅ¡ek WichterlovÃ¡', 144.54000000000002)]

In [20]:
# Q6) Write a query to return the email, first name, last name and genre of all rock music listeners.
# Return your list alphabetically by email starting with A.
cur.execute('''select distinct email, concat(first_name, " ", last_name) as full_name from customer join invoice on 
customer.customer_id = invoice.customer_id join invoice_line on invoice.invoice_id = invoice_line.invoice_id where track_id in 
(select track_id from track join genre on track.genre_id=genre.genre_id where genre.name like "Rock") order by email''')
cur.fetchall()

[('aaronmitchell@yahoo.ca', 'Aaron Mitchell'),
 ('alero@uol.com.br', 'Alexandre Rocha'),
 ('astrid.gruber@apple.at', 'Astrid Gruber'),
 ('bjorn.hansen@yahoo.no', 'BjÃ¸rn Hansen'),
 ('camille.bernard@yahoo.fr', 'Camille Bernard'),
 ('daan_peeters@apple.be', 'Daan Peeters'),
 ('diego.gutierrez@yahoo.ar', 'Diego GutiÃ©rrez'),
 ('dmiller@comcast.com', 'Dan Miller'),
 ('dominiquelefebvre@gmail.com', 'Dominique Lefebvre'),
 ('edfrancis@yachoo.ca', 'Edward Francis'),
 ('eduardo@woodstock.com.br', 'Eduardo Martins'),
 ('ellie.sullivan@shaw.ca', 'Ellie Sullivan'),
 ('emma_jones@hotmail.com', 'Emma Jones'),
 ('enrique_munoz@yahoo.es', 'Enrique MuÃ±oz'),
 ('fernadaramos4@uol.com.br', 'Fernanda Ramos'),
 ('fharris@google.com', 'Frank Harris'),
 ('fralston@gmail.com', 'Frank Ralston'),
 ('frantisekw@jetbrains.com', 'FrantiÅ¡ek WichterlovÃ¡'),
 ('ftremblay@gmail.com', 'FranÃ§ois Tremblay'),
 ('fzimmermann@yahoo.de', 'Fynn Zimmermann'),
 ('hannah.schneider@yahoo.de', 'Hannah Schneider'),
 ('hholy@gma

In [21]:
#Q7 Let's invite the artists who have written the most rock music in our dataset. Write a query that returns the Artist name 
# and total track count of the top 10 rock bands.
cur.execute('''select artist.artist_id,artist.name,count(track.track_id) as track_count from artist join album 
on artist.artist_id=album.artist_id join track on album.album_id=track.album_id join genre
on track.genre_id=genre.genre_id where genre.name like "Rock" group by artist.artist_id order by track_count desc limit 10''')
cur.fetchall()

[(1, 'AC/DC', 18),
 (3, 'Aerosmith', 15),
 (8, 'Audioslave', 14),
 (22, 'Led Zeppelin', 14),
 (4, 'Alanis Morissette', 13),
 (5, 'Alice In Chains', 12),
 (23, 'Frank Zappa & Captain Beefheart', 9),
 (2, 'Accept', 4)]

In [22]:
#Q8 Return all the track names that have a song length longer than the average song length. Return the Name and Milliseconds
# for each track. Order by the song length with the longest songs listed first.
cur.execute('''select name,milliseconds from track where milliseconds > (select avg(milliseconds) from track) 
order by milliseconds desc''')
cur.fetchall()

[('How Many More Times', 711836),
 ('Advance Romance', 677694),
 ('Sleeping Village', 644571),
 ('You Shook Me(2)', 619467),
 ("Talkin' 'Bout Women Obviously", 589531),
 ('Stratus', 582086),
 ('No More Tears', 555075),
 ('The Alchemist', 509413),
 ('Wheels Of Confusion / The Straightener', 494524),
 ('Book Of Thel', 494393),
 ('You Oughta Know (Alternate)', 491885),
 ('Terra', 482429),
 ("Snoopy's search-Red baron", 456071),
 ('Sozinho (Hitmakers Classic Mix)', 436636),
 ('Master Of Puppets', 436453),
 ('Stone Crazy', 433397),
 ('Snowblind', 420022),
 ('Computadores Fazem Arte', 404323),
 ('Jerusalem', 402390),
 ('Dazed and Confused', 401920),
 ('The Winner Loses', 392254),
 ('Love, Hate, Love', 387134),
 ('ConstruÃ§Ã£o / Deus Lhe Pague', 383059),
 ('Black Sabbath', 382066),
 ("Livin' On The Edge", 381231),
 ("I Can't Quit You Baby(2)", 380551),
 ('Princess of the Dawn', 375418),
 ('Harvester Of Sorrow', 374543),
 ('Whole Lotta Love', 373394),
 ("Momma's Gotta Die Tonight", 371539),
 (

In [26]:
#Q9Find how much amount spent by each customer on artists?Write a query to return customer name, artist name and total spent
cur.execute('''select c.customer_id,concat(c.first_name, " ", last_name) as full_name, bsa.artist_name,
            sum(il.unit_price*il.quantity)
            as amount_spent from invoice as i
			join customer as c on i.customer_id=c.customer_id join invoice_line as il on il.invoice_id=i.invoice_id join track 
            as t on t.track_id=il.track_id join album as alb
            on alb.album_id=t.album_id join (select concat(c.first_name, " ",c.last_name) as full_name,
            a.name as artist_name,
            a.artist_id as artistID,
            sum(il.unit_price*il.quantity) as total_sale
            from artist as a 
            join album as al on a.artist_id=al.artist_id 
            join track on track.album_id=al.album_id
            join invoice_line as il on track.track_id=il.track_id 
            join invoice on invoice.invoice_id=il.invoice_id 
            join customer as c on c.customer_id=invoice.customer_id 
            group by 3 order by 4 desc limit 1) as bsa on bsa.artistID=alb.artist_id group by 1,2,3 order 
            by 4 desc
            ''')
cur.fetchall()

[(54, 'Steve Murray', 'AC/DC', 17.82),
 (53, 'Phil Hughes', 'AC/DC', 10.89),
 (21, 'Kathy Chase', 'AC/DC', 10.89),
 (49, 'StanisÅ‚aw WÃ³jcik', 'AC/DC', 9.9),
 (1, 'LuÃ\xads GonÃ§alves', 'AC/DC', 7.920000000000001),
 (24, 'Frank Ralston', 'AC/DC', 7.920000000000001),
 (31, 'Martha Silk', 'AC/DC', 3.96),
 (16, 'Frank Harris', 'AC/DC', 2.9699999999999998),
 (42, 'Wyatt Girard', 'AC/DC', 2.9699999999999998),
 (6, 'Helena HolÃ½', 'AC/DC', 2.9699999999999998),
 (38, 'Niklas SchrÃ¶der', 'AC/DC', 2.9699999999999998),
 (35, 'Madalena Sampaio', 'AC/DC', 2.9699999999999998),
 (44, 'Terhi HÃ¤mÃ¤lÃ¤inen', 'AC/DC', 2.9699999999999998),
 (9, 'Kara Nielsen', 'AC/DC', 1.98),
 (34, 'JoÃ£o Fernandes', 'AC/DC', 1.98),
 (57, 'Luis Rojas', 'AC/DC', 1.98),
 (27, 'Patrick Gray', 'AC/DC', 1.98),
 (20, 'Dan Miller', 'AC/DC', 1.98),
 (30, 'Edward Francis', 'AC/DC', 1.98),
 (5, 'FrantiÅ¡ek WichterlovÃ¡', 'AC/DC', 1.98),
 (47, 'Lucas Mancini', 'AC/DC', 0.99),
 (43, 'Isabelle Mercier', 'AC/DC', 0.99),
 (19, 'Tim Go

In [ ]:
# Q10 We want to find out the most popular music Genre for each country. We determine the most popular genre as the genre with
# the highest amount of purchases. Write a query that returns each country along with the top Genre.
# For countries where the maximum number of purchases is shared return all Genres
cur.execute()
cur.fetchall()